In [ ]:
# Imports
%load_ext autoreload
%autoreload 2

import math
import matplotlib 
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy
from scipy.integrate import trapz
from scipy.integrate import cumtrapz
from scipy.special import erfinv
from tqdm import tqdm
import sys

from astropy.coordinates import Distance
import bilby

# Add module
sys.path.append('../../modules')
from compute_omega import sample_dict_compute_injected_omega
from gwBackground_module import *
from pygwb.detector import Interferometer
from pygwb.network import Network
from pygwb.parameters import Parameters
from pygwb.baseline import Baseline
from pygwb.simulator import Simulator

from constants import *
import equations
import redshiftPrior

In [ ]:
# Plot formatting
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
fontsize = 9
params = {
   'axes.labelsize': fontsize,
   'font.size': fontsize,
   'legend.fontsize': 8,
   'xtick.labelsize': fontsize,
   'ytick.labelsize': fontsize,
   'axes.titlesize':fontsize,
   'lines.linewidth':1,  
   'xtick.direction':'in',
   'ytick.direction':'in',
   'font.family':'Serif',
   'font.serif':'Hoefler Text',
   'axes.grid':True,
   'figure.figsize': (6.75, 4),
   'figure.dpi':250,
   'mathtext.fontset':'cm'
}

for param in params.keys():
    matplotlib.rcParams[param] = params[param]  

In [ ]:
# Constants
# Mass Parameters
BBH_min = 5 # minimum BBH mass
BBH_max = 44 # maximum BBH mass

# Frequency Parameters
fmax = 3000
fref = 25

# Data Simulation Parameters
T_obs = (7/365.25) # observation time (in yrs)
duration = T_obs*60*60*24*365.25 # duration of each segment of data (in s)
N_segs = 1 # number of data segments to generate
fs = 4096 # sampling rate (in Hz)
num_iterations = 12 # number of iterations to average Omega(f) over

In [ ]:
# Standard Prior Dictionary
priors = bilby.gw.prior.BBHPriorDict(aligned_spin=True)
priors['mass_1'] = bilby.core.prior.PowerLaw(alpha=-2.3, minimum=BBH_min, maximum=BBH_max)
priors['mass_ratio'] = bilby.core.prior.PowerLaw(alpha=1.5, minimum=0, maximum=1)
priors['theta_jn'] = bilby.core.prior.Uniform(minimum=0, maximum=2*np.pi, name='theta_jn')
priors['chi_1'] = 0
priors['chi_2'] = 0
priors['redshift'] = redshiftPrior.BrokenPowerLawRedshiftPrior(R0=R0, alpha=alpha, beta=beta, zp=zp, minimum=0, maximum=z_max, name='redshift')
priors['geocent_time'] = bilby.core.prior.Uniform(minimum=0, maximum=T_obs, name='geocent_time')
del priors['luminosity_distance']
del priors['chirp_mass']

In [ ]:
# Calculate number of injections
def calculate_num_injections(T_obs, priors):
    zs, p_dz = priors['redshift']._get_redshift_arrays()
    p_dz_centers = (p_dz[1:] + p_dz[:-1])/2.
    total_sum = np.sum(np.diff(zs) * p_dz_centers)
    N = T_obs * total_sum
    print(p_dz)
    return N

zs, p_dz = priors['redshift']._get_redshift_arrays()
N = calculate_num_injections(T_obs, priors)
print(N)

In [ ]:
# Weighted Method (Waveform Generator)
# Uniform Prior Dictionary
uniform_priors = bilby.gw.prior.BBHPriorDict(aligned_spin=True)
uniform_priors['mass_1'] = bilby.core.prior.Uniform(minimum=BBH_min, maximum=BBH_max, name='mass_1_uniform')
uniform_priors['mass_ratio'] = bilby.core.prior.Uniform(minimum=0, maximum=1, name='mass_ratio_uniform')
uniform_priors['redshift'] = bilby.core.prior.Uniform(minimum=0, maximum=z_max, name='redshift_uniform')

# Draw samples
N_inj = np.random.poisson(N.value)
injections = priors.sample(N_inj)

m1s = injections['mass_1']
qs = injections['mass_ratio']
zs = injections['redshift']
m2s = []
Mtots = [] # m1 + m2
Ms = [] # chirp masses
for i in range(len(injections['mass_1'])):
    m2s.append(equations.calculate_m2(m1s[i], qs[i]))
    Mtots.append(equations.calculate_Mtot(m1s[i], m2s[i]))
    Ms.append(equations.calculate_M(m1s[i], m2s[i]))

# Set up waveform generator
# Waveform generator parameters
sampling_frequency=4096
approximant='IMRPhenomD'

# Initialize waveform generator
waveform_generator = bilby.gw.WaveformGenerator(
    duration=2,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments={
        "waveform_approximant": approximant,
        "reference_frequency": 50,
        "minimum_frequency": 1
    }
)

# Calculate non-weighted dEdf from uniform distributions
freqs_W = waveform_generator.frequency_array
OmegaGW_W = np.zeros(len(freqs_W))

dEdfs = []
for i in range(N_inj):
    inj_params = {}
    # Generate the individual parameters dictionary for each injection
    for k in injections.keys():
        if k == 'signal_type':
            continue
        try:
            inj_params[k] = injections[k]['content'][i]
        except:
            inj_params[k] = injections[k][i]

    # Get frequency domain waveform
    polarizations = waveform_generator.frequency_domain_strain(inj_params)

    # Final PSD of the injection
    dEdf = np.abs(polarizations['plus']) ** 2 + np.abs(polarizations['cross']) ** 2
    dEdfs.append(dEdf)
    
# Calculate Omega(f)
dEdf_sum = 0
for i in range(N_inj):
    z = zs[i]
    m1 = m1s[i]
    m2 = m2s[i]
    q = qs[i]
    Mtot = Mtots[i]
    M = Ms[i]

    # Probability of drawing {z, m1, m2}
    p_z = priors['redshift'].prob(z)
    p_m1 = priors['mass_1'].prob(m1)
    p_q = priors['mass_ratio'].prob(q)

    pdraw_z = uniform_priors['redshift'].prob(z)
    pdraw_m1 = uniform_priors['mass_1'].prob(m1)
    pdraw_q = uniform_priors['mass_ratio'].prob(q)

    # Weight calculation
    r = equations.R_SI(alpha, beta, z, zp, R0)
    h = equations.Hubble_rate_SI(z, H0, omegaR, omegaM, omegak, omegaL)
    h0 = equations.Hubble_rate_SI(0, H0, omegaR, omegaM, omegak, omegaL)

    wi = (p_z/pdraw_z) * (p_m1/pdraw_m1) * (p_q/pdraw_q)
    dEdf = wi*dEdfs[i]

    dEdf_sum += dEdf
    
    # Add to Omega_spectrum
    # OmegaGW_W += 2 * np.pi**2 * freqs_W**3 * dEdf / (3 * h0**2)
OmegaGW_W = 2 * np.pi**2 * freqs_W**3 * dEdf_sum / (3 * h0**2)
Tobs_seconds = T_obs * 86400 * 365.25  # years to seconds
OmegaGW_W *= 2 / Tobs_seconds

In [ ]:
freqs_psd, omega_gw_freq, injections = sample_dict_compute_injected_omega(priors, Tobs=(7/365.25), duration=2, f_ref=25, sampling_frequency=4096,
                                       approximant='IMRPhenomD')

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
ax.loglog(freqs_W, OmegaGW_W, color='#bc85ff')
ax.loglog(freqs_psd, omega_gw_freq, color='#000000')
ax.set_title(r'GW Energy Density Spectrum (Weighted Method)')
ax.set_xlabel(r'Frequency (Hz)')
ax.set_ylabel(r'$\Omega_{GW}(f)$')
ax.set_xlim(10, 4096)
ax.legend()